In [44]:
import pickle
import json

In [45]:
src_train = pickle.load(open('../data/encoded_queries_train.pkl', 'rb'))
src_dev = pickle.load(open('../data/encoded_queries_dev.pkl', 'rb'))

raw_websites = json.load(open('../data/RC_2009-05_webpages.json', 'r'))


corpus_text = []
with open('../data/websites/bow_websites.jsonl', 'r') as f:
    for line in f:
        corpus_text.append(json.loads(line))

src_dev_text = open('../data/bow_queries_dev.tsv', 'r').read().splitlines()


corpus = pickle.load(open('../data/encoded_websites.pkl', 'rb'))


relevance_scores = open('../data/relevance_scores.txt', 'r')
query_webpage_map = {}
for line in relevance_scores:
    split_line = line.split()
    query_webpage_map[int(split_line[0])] = int(split_line[2])


In [46]:
src_dev[0]

{'query_id': 5335,
 'encoding': tensor([[ 0.0080,  0.0120, -0.0139,  ..., -0.0024,  0.0485, -0.0249],
         [ 0.0336,  0.0337,  0.0075,  ...,  0.0383,  0.0492,  0.0096]],
        device='cuda:0')}

In [47]:
src_dev_text[0]

'5335\twhat a world it would be without Murdock. [Murdock]( )[Murdoch](http://en.wikipedia.org/wiki/Rupert_Murdoch)'

In [49]:
query_webpage_map[5335]

200

In [50]:
print(corpus_text[200])

{'id': 200, 'contents': 'List of characters from the American TV show The A-TeamThis article lists the characters created for the popular 1980s American action-adventure television series The A-Team and its 2010 film adaptation as well as other media appearances.The A-Team. Clockwise from top: Murdock, B. A. Baracus, Hannibal and Faceman.George Peppard as Ex-United States Army Special Forces Commando Lieutenant-Colonel/Colonel John "Hannibal" Smith (1983–1987)John "Hannibal" Smith is the leader of the A-Team. He is a brilliant tactician and a master of disguise. Hannibal is distinguished by his cigar smoking, black gloves, disguises, and his catch phrase, "I love it when a plan comes together." Always "on the jazz" (a phrase coined by B.A., meaning that Hannibal thrives on adventure and life-threatening situations), he seems to genuinely enjoy every situation they get into. He also works as an actor, playing monsters in low-budget horror movies. He is portrayed by Liam Neeson in the fi

In [51]:
print(len(corpus))
print(len(corpus_text))

5953
5953
